## NoSQL (MongoDB) (sesión 2), 25-26.

## Introducción

![MongoDB](https://webassets.mongodb.com/_com_assets/cms/MongoDB_Logo_FullColorBlack_RGB-4td3yuxzjs.png)

Esta hoja muestra cómo acceder a bases de datos MongoDB y también a conectar la salida con Jupyter. Se puede utilizar el *shell* propio de MongoDB en el contenedor usando el programa `mongo`. La diferencia es que ese programa espera código Javascript y aquí trabajaremos con Python.

In [1]:
import sys

RunningInCOLAB: bool = 'google.colab' in sys.modules

## Instalación inicial de MongoDB (sólo necesaria en Google Colab)

In [2]:
import subprocess
import urllib.request

if RunningInCOLAB:
  urllib.request.urlretrieve("https://raw.githubusercontent.com/dsevilla/bdge/refs/heads/25-26/addendum/mongo-utils/run-mongo-local.sh",
                             "run-mongo-local.sh")
  subprocess.run(["sh", "run-mongo-local.sh"])

## Instalación de la librería `pymongo`

In [3]:
%pip install --upgrade pymongo 'pandas[performance,parquet]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.1 which is incompatible.
db-dtypes 1.5.0 requires pandas<3.0.0,>=1.5.3, but you have pandas 3.0.1 which is incompatible.
bqplot 0.12.45 requires pandas<3.0.0,>=1.0.0, but you have pandas 3.0.1 which is incompatible.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.1 which 

In [4]:
from pprint import pprint as pp

import pandas as pd

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [5]:
from pymongo import MongoClient

## Conexión a la base de datos

La conexión se inicia con `MongoClient` en el `host` descrito en el fichero `docker-compose.yml` (establecido en la variable de entorno `DB_HOSTNAME`), o bien a `localhost` si lo estamos haciendo en Colab.

In [6]:
import os

db_hostname: str = os.getenv('DB_HOSTNAME', "localhost")

In [7]:
from typing import Any

client: MongoClient[dict[str, Any]] = MongoClient(db_hostname,27017,compressors='zlib')
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, compressors=['zlib'])

In [8]:
client.list_database_names()

['admin', 'config', 'local']

Las bases de datos se crean conforme se nombran. Se puede utilizar la notación punto o la de diccionario. Las colecciones también se crean implícitamente al insertar un primer documento, o las podemos crear explícitamente.

In [9]:
from pymongo.collection import Collection
from pymongo.database import Database

db: Database = client.stackoverflow
# db: Database = client['stackoverflow'] # (equivalente)
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, compressors=['zlib']), 'stackoverflow')

## Importación de los datos de Stackoverflow

Procedemos a importar los ficheros Parquet, pero antes ponemos el formato y signifcado de los campos de `Stack Overflow` para poder consultarlos si lo necesitamos.

<details><summary><b>🗂️ Formato datos Stack Overflow</b> <span style="color: #007acc;">▶️ Haz click para ver el esquema y ejemplos</span></summary>

> Notas generales
- Formato: cada fichero es XML y suele distribuirse 7zipped.
- Codificación: UTF-8. Los campos de fecha siguen el formato ISO8601: "YYYY-MM-DDTHH:mm:ss.fff".
- Nulos: cuando un campo no aplica (p.ej., `ParentId` en preguntas), no aparece o viene vacío; en nuestros Parquet suele ser `null`.
- Identificadores: `Id` es clave primaria en cada entidad. Claves foráneas enlazan por `Id` entre ficheros.

---

:file_folder: <b>badges.xml</b> (insignias logradas por usuarios)
- UserId: Id del usuario que recibe la insignia. Ej: 420. Uso: FK a `users.Id`.
- Name: Nombre de la insignia. Ej: "Teacher". Uso: clasificar tipo de logro.
- Date: Fecha de concesión. Ej: "2008-09-15T08:55:03.923". Uso: series temporales.
- Class: Categoría (1=Gold, 2=Silver, 3=Bronze). Ej: 3. Uso: nivel de la insignia.
- TagBased: Si la insignia es específica de etiqueta. Ej: true. Uso: filtrar logros por etiquetas.

---

:file_folder: <b>comments.xml</b> (comentarios en publicaciones)
- Id: Identificador del comentario. Ej: 12345. Uso: PK.
- PostId: Publicación a la que comenta. Ej: 100234. Uso: FK a `posts.Id`.
- Score: Puntuación del comentario. Ej: 5. Uso: ordenar/filtrar relevancia.
- Text: Texto del comentario. Ej: "¿Puedes compartir el error exacto?". Uso: contenido.
- CreationDate: Fecha de creación. Ej: "2008-09-06T08:07:10.730". Uso: análisis temporal.
- UserId: Autor del comentario. Ej: 314159. Uso: FK a `users.Id`.
- UserDisplayName: Nombre mostrado si el usuario fue eliminado. Ej: "user123". Uso: se usa cuando `UserId` es nulo/ausente.
- ContentLicense: Licencia del contenido. Ej: "CC BY-SA 4.0". Uso: cumplimiento/licencias.

---

:file_folder: <b>posts.xml</b> (preguntas y respuestas)
- Id: Identificador de la publicación. Ej: 98765. Uso: PK.
- PostTypeId: Tipo (1=Pregunta, 2=Respuesta). Ej: 1. Uso: rutas de procesamiento/particionado.
- ParentId: Id de la pregunta padre si es respuesta. Ej: 1234. Uso: FK a `posts.Id` (solo si `PostTypeId=2`).
- AcceptedAnswerId: Id de la respuesta aceptada (solo en preguntas). Ej: 4567. Uso: FK a `posts.Id` para marcar aceptadas.
- CreationDate: Fecha de creación. Ej: "2010-02-15T10:21:34.120". Uso: análisis temporal.
- Score: Votos netos (upvotes-downvotes). Ej: 42. Uso: ranking/calidad.
- ViewCount: Nº de vistas (solo preguntas). Ej: 12345. Uso: popularidad.
- Body: HTML/Markdown del cuerpo. Ej: "<p>¿Cómo...?</p>". Uso: contenido/NLP.
- OwnerUserId: Autor. Ej: 271828. Uso: FK a `users.Id`.
- OwnerDisplayName: Nombre mostrado si el autor fue eliminado. Ej: "John Doe". Uso: se usa cuando `OwnerUserId` es nulo (usuario eliminado o anónimo).
- LastEditorUserId: Último usuario que editó. Ej: 314159. Uso: FK a `users.Id`.
- LastEditorDisplayName: Nombre si el editor fue eliminado. Ej: "Community". Uso: solo cuando `LastEditorUserId` es nulo.
- LastEditDate: Fecha de última edición. Ej: "2009-03-05T22:28:34.823". Uso: auditoría.
- LastActivityDate: Última actividad (edición, comentario, etc.). Ej: "2009-03-11T12:51:01.480". Uso: ordenación por actividad reciente.
- CommunityOwnedDate: Fecha desde la que es de la comunidad. Ej: "2009-03-11T12:51:01.480". Uso: gobernanza.
- ClosedDate: Fecha de cierre (si procede). Ej: "2012-01-01T12:00:00.000". Uso: moderación.
- Title: Título de la pregunta. Ej: "¿Cómo unir listas en Python?". Uso: metadata/búsqueda.
- Tags: Lista de etiquetas en formato XML/HTML: "<python><list><merge>". Uso: clasificación por tema.
- AnswerCount: Nº de respuestas (en preguntas). Ej: 5. Uso: engagement.
- CommentCount: Nº de comentarios. Ej: 3. Uso: actividad.
- FavoriteCount: Favoritos marcados (legacy). Ej: 10. Uso: popularidad histórica.
- ContentLicense: Licencia. Ej: "CC BY-SA 4.0". Uso: cumplimiento.

---

:file_folder: <b>posthistory.xml</b> (historial detallado de cambios)
- Id: PK del evento de historial. Ej: 5555.
- PostHistoryTypeId: Tipo de cambio. Uso: clasifica el evento. Ejemplos:
  - 1: Initial Title – primer título de la pregunta.
  - 2: Initial Body – primer cuerpo en crudo.
  - 3: Initial Tags – primeras etiquetas.
  - 4: Edit Title – cambio de título.
  - 5: Edit Body – cambio de cuerpo (markdown crudo).
  - 6: Edit Tags – cambio de etiquetas.
  - 7/8/9: Rollback Title/Body/Tags – revertidos.
  - 10: Post Closed – cierre por votos.
  - 11: Post Reopened – reapertura.
  - 12/13: Post Deleted/Undeleted – borrado/restaurado.
  - 14/15: Post Locked/Unlocked – bloqueado/desbloqueado.
  - 16: Community Owned – pasa a comunidad.
  - 17: Post Migrated – migración (origen/destino).
  - 18: Question Merged – fusión de preguntas.
  - 19/20: Question Protected/Unprotected – protegido/desprotegido.
  - 21: Post Disassociated – se elimina el OwnerUserId.
  - 22: Question Unmerged – deshace fusión.
- PostId: Publicación afectada. Ej: 98765. Uso: FK a `posts.Id`.
- RevisionGUID: Agrupa múltiples registros de un mismo acto. Ej: "3E5B...". Uso: correlación.
- CreationDate: Fecha del evento. Ej: "2009-03-05T22:28:34.823".
- UserId: Usuario que realiza el cambio. Ej: 271828. Uso: FK a `users.Id`.
- UserDisplayName: Si el usuario fue eliminado. Ej: "user123". Uso: cuando `UserId` nulo.
- Comment: Comentario del editor. Ej: "typo fix". Uso: auditoría.
- Text: Valor crudo nuevo asociado al cambio. Ej: nuevo markdown o JSON.
  - Para tipos 10–15: JSON con usuarios que han votado esa acción de moderación.
  - Para tipo 17: Detalles de migración: "from <url>" o "to <url>".
- CloseReasonId: Motivo de cierre (histórico). Ejemplos:
  - 1: Exact Duplicate – duplicada de otra.
  - 2: off-topic – fuera de tema.
  - 3: subjective – demasiado subjetiva.
  - 4: not a real question – no es una pregunta real.
  - 7: too localized – demasiado localizada.

---

:file_folder: <b>postlinks.xml</b> (enlaces entre publicaciones)
- Id: PK del enlace. Ej: 2222.
- CreationDate: Fecha del enlace. Ej: "2011-05-05T10:00:00.000".
- PostId: Publicación origen. Ej: 1000. Uso: FK a `posts.Id`.
- RelatedPostId: Publicación destino. Ej: 1001. Uso: FK a `posts.Id`.
- PostLinkTypeId: Tipo de relación. Uso: grafo entre posts.
  - 1: Linked – relacionados.
  - 3: Duplicate – duplicados.

---

:file_folder: <b>users.xml</b> (usuarios)
- Id: PK del usuario. Ej: 271828.
- AccountId: Id de cuenta en la red Stack Exchange. Ej: 123456. Uso: consolidación multi-sitio.
- Reputation: Reputación. Ej: 15234. Uso: privilegios/ordenación.
- CreationDate: Alta del usuario. Ej: "2008-07-31T21:42:52.667".
- DisplayName: Nombre mostrado. Ej: "Jane Doe". Uso: UI.
- EmailHash: Hash MD5 de email (legacy). Ej: "fcea920f...". Uso: avatar histórico.
- LastAccessDate: Último acceso. Ej: "2020-04-01T12:00:00.000". Uso: actividad.
- WebsiteUrl: Web personal. Ej: "https://janedoe.dev".
- Location: Ubicación libre. Ej: "Madrid, ES".
- Age: Edad (cuando se compartía). Ej: 32.
- AboutMe: BIO en HTML/Markdown. Ej: "<p>Desarrollo datos...</p>".
- Views: Visitas al perfil. Ej: 1234.
- UpVotes: Upvotes emitidos por el usuario. Ej: 500.
- DownVotes: Downvotes emitidos. Ej: 20.
- ProfileImageUrl: Avatar. Ej: "https://.../image.png".

---

:file_folder: <b>votes.xml</b> (votos a publicaciones)
- Id: PK del voto. Ej: 7777.
- PostId: Publicación votada. Ej: 98765. Uso: FK a `posts.Id`.
- VoteTypeId: Tipo de voto/acción. Uso: métricas de calidad/moderación.
  - 1: AcceptedByOriginator – el autor de la pregunta acepta una respuesta (equivale a `AcceptedAnswerId`).
  - 2: UpMod – upvote.
  - 3: DownMod – downvote.
  - 4: Offensive – ofensivo (legacy).
  - 5: Favorite – marcado como favorito (legacy); si `VoteTypeId=5`, `UserId` se rellena.
  - 6: Close – voto de cierre.
  - 7: Reopen – voto de reapertura.
  - 8: BountyStart – inicio de recompensa.
  - 9: BountyClose – cierre de recompensa; si `VoteTypeId=9`, `BountyAmount` se rellena.
  - 10: Deletion – voto de borrado.
  - 11: Undeletion – voto de restauración.
  - 12: Spam – marcado como spam.
  - 13: InformModerator – informar a moderación.
- CreationDate: Fecha del voto. Ej: "2015-06-10T09:30:00.000".
- UserId: Usuario que vota (solo para algunos tipos como 5). Ej: 54321. Uso: FK a `users.Id`.
- BountyAmount: Cantidad de recompensa (solo tipo 9). Ej: 100.

---

:file_folder: <b>tags.xml</b> (etiquetas del sitio)
- Id: PK de la etiqueta. Ej: 42.
- TagName: Nombre de la etiqueta. Ej: "python". Uso: clasificación temática.
- Count: Nº de usos en preguntas. Ej: 150234. Uso: popularidad.
- ExcerptPostId: Id del post con el extracto de la wiki de etiqueta. Ej: 123. Uso: FK a `posts.Id`.
- WikiPostId: Id del post con el artículo de la wiki de etiqueta. Ej: 124. Uso: FK a `posts.Id`.

</details>

Importamos los ficheros Parquet, primero definiendo las funciones que se necesitan, y después ejecutándolas.

In [10]:
from asyncio import Queue, Task, create_task, to_thread

from pandas import DataFrame


def load_dataframe(parquetfile: str, logqueue: Queue[str] | None = None) -> DataFrame:
    BASE_URL: str = "https://github.com/dsevilla/bd2-data/releases/download/parquet-files-25-26/"
    try:
        df: DataFrame = pd.read_parquet(f"{BASE_URL}/{parquetfile}", dtype_backend="pyarrow")
        logqueue.put_nowait(f"Loaded {len(df)} rows from dataframe.") if logqueue else None

        return df

    except Exception as e:
        logqueue.put_nowait(f"Error loading Dataframe: {e}.") if logqueue else None
        raise e

In [11]:
from pymongo.results import InsertManyResult


def insert_dataframe_into_mongo(
    collection: Collection, df: DataFrame, logqueue: Queue | None = None
) -> InsertManyResult:
    result: InsertManyResult = collection.insert_many(
        df.to_dict("records"), ordered=False, bypass_document_validation=True
    )
    logqueue.put_nowait(f"Inserted {len(df)} records into MongoDB.") if logqueue else None
    return result


def load_parquet_into_mongo(
    collection: Collection,
    parquetfile: str,
    logqueue: Queue[str] | None = None,
) -> InsertManyResult:
    df: DataFrame = load_dataframe(parquetfile, logqueue)
    try:
        return insert_dataframe_into_mongo(collection, df, logqueue)
    except Exception as e:
        logqueue.put_nowait(f"Error inserting DataFrame into MongoDB: {e}.") if logqueue else None
        raise e
    finally:
        del df  # Free memory

Importación de los ficheros Parquet. Por ahora creamos una colección diferente para cada uno. Después estudiaremos cómo poder optimizar el acceso usando agregación.

Creamos un índice para el campo 'Id' en la colección de tags.

In [12]:
tags_collection: Collection[dict[str, Any]] = db["tags"]
tags_collection.create_index([("Id", 1)], unique=True)

'Id_1'

Se crea una cola de mensajes (`Queue`) para que los procesos en segundo plano puedan ir mostrando el progreso de la importación. En cualquier momento se puede leer mensajes de esa cola (si están disponibles) y podemos ver si se ha producido algún error o si se ha terminado de procesar los elementos.

La importación de los *Tags* se realiza en primer plano porque es pequeña. El resto de colecciones las importaremos en segundo plano (con un hilo o *thread*).

In [13]:
# No añadir datos si ya existe la colección
existing_tags_count: int = tags_collection.count_documents({})
if existing_tags_count > 0:
    print("La colección Tags ya existe. No se realiza la importación. Elimínala con drop().")
else:
    insert_tags_queue: Queue[str] = Queue()
    load_parquet_into_mongo(tags_collection, "Tags.parquet", insert_tags_queue)
    while not insert_tags_queue.empty():
        log_message: str = await insert_tags_queue.get()
        print(log_message)

Loaded 3048 rows from dataframe.
Inserted 3048 records into MongoDB.


In [14]:
tags_collection.find_one()

{'_id': ObjectId('69a0011b6103b90d71d66187'),
 'Id': 1,
 'Count': 1472,
 'ExcerptPostId': 680,
 'TagName': '.net',
 'WikiPostId': 679}

Importamos los datos de Stackoverflow. Lo haremos en paralelo con todas las colecciones restantes:

In [15]:
users_collection: Collection[dict[str, Any]] = db["users"]
users_collection.create_index([("Id", 1)], unique=True)

users_load_task: Task[InsertManyResult] = create_task(
    to_thread(load_parquet_into_mongo, users_collection, "Users.parquet")
)

In [16]:
posts_collection: Collection[dict[str, Any]] = db["posts"]
posts_collection.create_index([("Id", 1)], unique=True)

posts_load_queue: Queue[str] = Queue()
posts_load_task: Task[InsertManyResult] = create_task(
    to_thread(load_parquet_into_mongo, posts_collection, "Posts.parquet", posts_load_queue)
)

In [17]:
comments_collection: Collection[dict[str, Any]] = db["comments"]
comments_collection.create_index([("Id", 1)], unique=True)

comments_load_queue: Queue[str] = Queue()
comments_load_task: Task[InsertManyResult] = create_task(
    to_thread(load_parquet_into_mongo, comments_collection, "Comments.parquet", comments_load_queue)
)

In [18]:
votes_collection: Collection[dict[str, Any]] = db["votes"]
votes_collection.create_index([("Id", 1)], unique=True)

votes_load_queue: Queue[str] = Queue()
votes_load_task: Task[InsertManyResult] = create_task(
    to_thread(load_parquet_into_mongo, votes_collection, "Votes.parquet", votes_load_queue)
)

In [19]:
print(f"Post loading done: {posts_load_task.done()}.")
while not posts_load_queue.empty():
    log_message: str = posts_load_queue.get_nowait()
    print(log_message)

Post loading done: False.


In [20]:
users_load_task.done()

False

In [21]:
comments_load_task.done()

False

In [22]:
votes_load_task.done()

False

## Framework de Agregación

Framework de agregación:
- Aquí está la referencia de las diferentes etapas por las que puede pasar un pipeline: https://www.mongodb.com/docs/manual/reference/operator/aggregation-pipeline/.
- Y aquí los distintos operadores que se permiten dentro de las etapas: https://docs.mongodb.com/manual/reference/operator/aggregation/.
- Y aquí incluso un libro completo con usos prácticos de ejecutar agregación: https://www.practical-mongodb-aggregations.com/.

A continuación un vídeo interesante:

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('VSX4a3h4SmQ',width=800)

Aquí tenemos un esquema con un ejemplo básico de agregación.

![](https://miro.medium.com/max/1060/1*2lDBxvZ8Cr3JYkoODTa0lQ.png)

- match : que queremos buscar
- group : como los queremos agrupar

## Esperamos a que todo se haya cargado para poder continuar.

In [23]:
import asyncio

await asyncio.gather(
    users_load_task,
    posts_load_task,
    comments_load_task,
    votes_load_task
);

## Algunos operadores de agregación

### `$match`

Este operador permite filtrar los documentos que queremos que pasen a la siguiente fase del pipeline definiendo una serie de condiciones sobre los campos de los documentos. Es equivalente al find. Si ponemos varios sin nada, actua como un and.

Vamos a filtrar aquellos documentos que tengan un `Score` igual o superior (`$gte`) a 150.

In [24]:
from pymongo.command_cursor import CommandCursor

respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
        {'$match': { 'Score' : {'$gte': 150}}}
])

# A diferencia del find (con sort, limit, explain..) aqui no los podemos usar

pp(list(respuestas))

[{'AcceptedAnswerId': None,
  'AnswerCount': 0,
  'Body': '<p>Asumiendo que se está haciendo un <code>join</code> de columnas '
          'sin duplicados, lo cuál es un caso '
          'común:</p><br/><ul><br/><li><p>Un <strong>inner join</strong> de A '
          'y B entregará el resultado de la intersección de los conjuntos A y '
          'B. En otras palabras, la parte <em>interna</em> –intersección– en '
          'un diagrama de Venn.</p><br/></li><br/><li><p>Un <strong>full outer '
          'join</strong> entre A y B entregará el resultado de la unión de A y '
          'B. En otras palabras, la parte <em>externa</em> –unión– en un '
          'diagrama de Venn '
          '.</p><br/></li><br/></ul><br/><h1>Ejemplos:</h1><br/><p>Supongamos '
          'que tenemos dos tablas, con una sola columna cada una y los '
          'siguientes datos:</p><br/><pre><code>A    B<br/>-    -<br/>1    '
          '3<br/>2    4<br/>3    5<br/>4    6<br/></code></pre><br/><p>Nota '
          

### `$project`

El operador `$project`permite filtar qué campos de los documentos queremos usar en la siguientes fases de agregación.

Generalmente este operador suele combinarse con otros como `$find`, `$match` o `$lookup` en fases más avanzadas de la agregación.

En el siguiente ejemplo, realizamos un filtrado en donde solo nos quedamos con el campo `Id` de los posts.

In [25]:
respuestas: CommandCursor[dict[str, Any]] = db['posts'].aggregate( [
    {'$project' : { 'Id' : True }},
    {'$limit': 20} ])
pp(list(respuestas))

[{'Id': 1, '_id': ObjectId('69a001966103b90d71f1d877')},
 {'Id': 2, '_id': ObjectId('69a001966103b90d71f1d878')},
 {'Id': 3, '_id': ObjectId('69a001966103b90d71f1d879')},
 {'Id': 4, '_id': ObjectId('69a001966103b90d71f1d87a')},
 {'Id': 5, '_id': ObjectId('69a001966103b90d71f1d87b')},
 {'Id': 6, '_id': ObjectId('69a001966103b90d71f1d87c')},
 {'Id': 7, '_id': ObjectId('69a001966103b90d71f1d87d')},
 {'Id': 8, '_id': ObjectId('69a001966103b90d71f1d87e')},
 {'Id': 9, '_id': ObjectId('69a001966103b90d71f1d87f')},
 {'Id': 10, '_id': ObjectId('69a001966103b90d71f1d880')},
 {'Id': 11, '_id': ObjectId('69a001966103b90d71f1d881')},
 {'Id': 12, '_id': ObjectId('69a001966103b90d71f1d882')},
 {'Id': 14, '_id': ObjectId('69a001966103b90d71f1d883')},
 {'Id': 15, '_id': ObjectId('69a001966103b90d71f1d884')},
 {'Id': 16, '_id': ObjectId('69a001966103b90d71f1d885')},
 {'Id': 17, '_id': ObjectId('69a001966103b90d71f1d886')},
 {'Id': 18, '_id': ObjectId('69a001966103b90d71f1d887')},
 {'Id': 19, '_id': Obje

### `$group`

La función `$group` se usa dentro del *pipeline* de agregación de documentos seguido por Mongo (`.aggregate`). Esta función admite dos parámetros diferentes:
* `_id`: El identificador por el que queremos agrupar los documentos.
* `campo`: Expresión mediante la cual queremos aggregar los documentos (esto es opcional).

Vamos a contar el número de posts por usuario, usando el campo `OwnerUserId`, en la colección `posts`.

In [27]:
from pymongo.command_cursor import CommandCursor

users_count_scores: CommandCursor[dict[str, Any]] = db.posts.aggregate(
    [
        {
            "$group":{
                "_id": "$OwnerUserId", # el identificador por el que quiero agrupar. Ahora el _id no es el object, va a mostrar el OwnerUserId
                "count": {"$sum": 1}  # el campo de lo que quiero hacer
            }
        },
        {
            "$limit": 10  #si lo hubiera puesto antes, hubiera cogido los 10 primeros y ya
        }
    ]
)
pp(list(users_count_scores))

[{'_id': 211181, 'count': 4},
 {'_id': 229627, 'count': 1},
 {'_id': 264363, 'count': 1},
 {'_id': 308557, 'count': 2},
 {'_id': 310708, 'count': 3},
 {'_id': 287198, 'count': 2},
 {'_id': 156913, 'count': 1},
 {'_id': 163196, 'count': 2},
 {'_id': 310960, 'count': 1},
 {'_id': 323950, 'count': 1}]


Ahora vamos calcular el `score` medio por usuario, usando de nuevo `OnwerUserId`, en base a todos sus posts.

In [28]:
users_avg_scores: CommandCursor[dict[str, Any]] = db.posts.aggregate(
    [{
        "$group":{
            "_id": "$OwnerUserId",
            "avg_score": {"$avg":'$Score'}
        }
        },
        { "$limit": 10 }
    ]
)
pp(list(users_avg_scores))

[{'_id': 211181, 'avg_score': 0.75},
 {'_id': 229627, 'avg_score': 2.0},
 {'_id': 264363, 'avg_score': 2.0},
 {'_id': 308557, 'avg_score': 0.5},
 {'_id': 310708, 'avg_score': 0.6666666666666666},
 {'_id': 287198, 'avg_score': -0.5},
 {'_id': 156913, 'avg_score': 0.0},
 {'_id': 163196, 'avg_score': 1.0},
 {'_id': 310960, 'avg_score': 0.0},
 {'_id': 323950, 'avg_score': 0.0}]


### `$lookup`

El operador `$lookup` permite realizar búsquedas en otras colecciones. Podrían interpretarse como un *join* en el modelo relacional. Esta función admite cuatro parámetros diferentes:

*   `from`: especifica la colección en la misma base de datos con la que hacer la búsqueda (o el *join*).
*   `localField`: Especifica el campo de los documentos de entrada a usar para realizar una comparación de igualdad con el campo `foreignField` de los documentos de la colección `from`.  
*   `foreignField`: Especifica el campo de los documentos de la colección `from` a usar en la comparación de igualdad con el `localField`.
*   `as`: Especifica el nombre del nuevo campo array que se añadirá a los documentos de entrada. Este nuevo campo contiene los documentos coincidentes de la colección `from`.

En el ejemplo siguiente, en primer lugar seleccionamos los posts con un `Score` igual o mayor a 150 con `$match` para a continuación seleccionar los usuarios que han publicado dichos posts incluyéndolo en un nuevo campo llamado `owner`.

In [29]:
respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
    {'$match': { 'Score' : {'$gte': 150}}},
    {'$lookup': {
        'from': "users",  #nombre de la coleccion DENTRO de db de la que me traigo los datos
        'localField': "OwnerUserId", #la llave ajena dentro de tu documento de posts
        'foreignField': "Id", #la correlacion que existe entre ellos. la "llave primaria" en la colección de users. Busca en la colección de usuarios a alguien cuyo Id coincida con el OwnerUserId del post
        'as': "owner"}  #nombre del nuevo campo que se añadirá a cada documento de post
    }
])
pp(list(respuestas))

[{'AcceptedAnswerId': None,
  'AnswerCount': 0,
  'Body': '<p>Asumiendo que se está haciendo un <code>join</code> de columnas '
          'sin duplicados, lo cuál es un caso '
          'común:</p><br/><ul><br/><li><p>Un <strong>inner join</strong> de A '
          'y B entregará el resultado de la intersección de los conjuntos A y '
          'B. En otras palabras, la parte <em>interna</em> –intersección– en '
          'un diagrama de Venn.</p><br/></li><br/><li><p>Un <strong>full outer '
          'join</strong> entre A y B entregará el resultado de la unión de A y '
          'B. En otras palabras, la parte <em>externa</em> –unión– en un '
          'diagrama de Venn '
          '.</p><br/></li><br/></ul><br/><h1>Ejemplos:</h1><br/><p>Supongamos '
          'que tenemos dos tablas, con una sola columna cada una y los '
          'siguientes datos:</p><br/><pre><code>A    B<br/>-    -<br/>1    '
          '3<br/>2    4<br/>3    5<br/>4    6<br/></code></pre><br/><p>Nota '
          

Aqui owner tendria la forma owner = [ {Juan, id=21}, {Pilar, id=21}...]

### `$arrayElemAt`

El `$lookup` genera un _array_ con todos los resultados. El operador `$arrayElemenAt` accede al primer elemento.

En este ejemplo, de nuevo seleccionamos, en primer lugar, los posts con un `Score` igual o mayor a 150 con `$match` para a continuación seleccionar los usuarios que han publicado dichos posts incluyéndolo en un nuevo campo llamado `owner`. Finalmente, seleccionamos sólo el elemento en la posición 0 del array generado para `username`.

In [30]:
respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
    {'$match': { 'Score' : {'$gte': 150}}},
    {'$lookup': {
        'from': "users",
        'localField': "OwnerUserId",
        'foreignField': "Id",
        'as': "owner"} # el nombre que le he dado en el 'join'
    },
    { '$project' : {
        'Id' : True,
        'Score' : True,
        'username' : {'$arrayElemAt' : ['$owner.DisplayName', 0]}, #en el array que te ha salido, coge el elemento 0 y de esos me coges ownerDisplayName
        'owner.DisplayName' : True #del owner, muestrame solo el campo DisplayName
      }}
    ])

# owner.DisplayName accede a los nombres dentro de esa lista.
# $arrayElemAt con el índice 0 extrae el primer elemento de esa lista y lo guarda en un campo nuevo y plano llamado username

pp(list(respuestas))

[{'Id': 40,
  'Score': 206,
  '_id': ObjectId('69a001966103b90d71f1d89b'),
  'owner': [{'DisplayName': 'jachguate'}],
  'username': 'jachguate'},
 {'Id': 197,
  'Score': 176,
  '_id': ObjectId('69a001966103b90d71f1d91b'),
  'owner': [{'DisplayName': 'Peter Olson'}],
  'username': 'Peter Olson'},
 {'Id': 204,
  'Score': 195,
  '_id': ObjectId('69a001966103b90d71f1d91f'),
  'owner': [{'DisplayName': 'Konamiman'}],
  'username': 'Konamiman'},
 {'Id': 18232,
  'Score': 232,
  '_id': ObjectId('69a001976103b90d71f210e9'),
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'},
 {'Id': 18233,
  'Score': 219,
  '_id': ObjectId('69a001976103b90d71f210ea'),
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'},
 {'Id': 87842,
  'Score': 173,
  '_id': ObjectId('69a001976103b90d71f2d9a0'),
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'}]


Aqui me sale como 'owner': [{'DisplayName': 'jachguate'}], en vez de tener que acceder a post['owner'][0]['DisplayName'], ahora puedes usar simplemente post['username'].

### `$unwind`

Este operador *desdobla* una fila por cada elemento del array de la colección.

Ej: El siguiente código:

```python
db.inventario.insert_one({ "_id" : 1, "item" : "ABC1", "tallas": [ "S", "M", "L"] })
db.inventario.aggregate( [ { "$unwind" : "$tallas" } ] )
```

Devolverá:

```json
{ "_id" : 1, "item" : "ABC1", "tallas" : "S" }
{ "_id" : 1, "item" : "ABC1", "tallas" : "M" }
{ "_id" : 1, "item" : "ABC1", "tallas" : "L" }
```

En el ejemplo de procesamiento de posts que estamos llevando a cabo, como sabemos que el array `$owner` sólo contiene un elemento, sólo habrá una fila por fila original, pero sin el _array_. Finalmente se puede proyectar el campo que se quiera. En este caso el `ownerDisplayName` que es proyectado (renombrado) como `username`.

In [31]:
respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
    { '$match': { 'Score' : {'$gte': 150}}},
    { '$lookup': {
        'from': "users",
        'localField': "OwnerUserId",
        'foreignField': "Id",
        'as': "owner"
      }
    },
    { '$unwind': '$owner'},
    { '$project' : {
          'username': '$owner.DisplayName'  #proyecta como el arrayelemetAt porque SOLO tenia un elemento. Si no, aparecería como el ejemplo.
      }
    }
    ])
pp(list(respuestas))

[{'_id': ObjectId('69a001966103b90d71f1d89b'), 'username': 'jachguate'},
 {'_id': ObjectId('69a001966103b90d71f1d91b'), 'username': 'Peter Olson'},
 {'_id': ObjectId('69a001966103b90d71f1d91f'), 'username': 'Konamiman'},
 {'_id': ObjectId('69a001976103b90d71f210e9'), 'username': 'Alvaro Montoro'},
 {'_id': ObjectId('69a001976103b90d71f210ea'), 'username': 'Alvaro Montoro'},
 {'_id': ObjectId('69a001976103b90d71f2d9a0'), 'username': 'Alvaro Montoro'}]


### `$push`

Devuelve un array con *todos* los valores que resultan de aplicar una determinada expresión a los documentos que forman parte del pipeline.

El siguiente ejemplo obtiene el listado con el identificador de `Posts` (`Id`) asociados a cada valor de `Score`.

In [32]:
posts_by_score: CommandCursor[dict[str, Any]] = db.posts.aggregate([
    { '$limit': 2000 },
    {
        '$group':{
            '_id': '$Score', #agrupalos por el score
            'posts':{           #Por cada documento que entra en un grupo, el operador $push añade un pequeño objeto { 'post': ID_DEL_POST } a una lista (array) llamada posts.
                '$push': {
                    'post': '$Id'
                }
            }
        }
    }

])
pp(list(posts_by_score))

[{'_id': 30,
  'posts': [{'post': 161},
            {'post': 424},
            {'post': 438},
            {'post': 454},
            {'post': 1653}]},
 {'_id': 54, 'posts': [{'post': 335}]},
 {'_id': 41,
  'posts': [{'post': 43}, {'post': 397}, {'post': 622}, {'post': 932}]},
 {'_id': 35, 'posts': [{'post': 309}, {'post': 560}]},
 {'_id': 0,
  'posts': [{'post': 14},
            {'post': 15},
            {'post': 19},
            {'post': 20},
            {'post': 21},
            {'post': 22},
            {'post': 85},
            {'post': 86},
            {'post': 98},
            {'post': 99},
            {'post': 113},
            {'post': 114},
            {'post': 115},
            {'post': 116},
            {'post': 124},
            {'post': 125},
            {'post': 128},
            {'post': 143},
            {'post': 144},
            {'post': 163},
            {'post': 164},
            {'post': 165},
            {'post': 166},
            {'post': 172},
            {'post

### `$addToSet`

 Devuelve un array de todos los valores únicos que resultan de aplicar una expresión a cada documento de un grupo.


Este ejemplo obtiene el listado de `Tags` asociadas a cada tipo de licencia  `ContentLicense`.

In [33]:
license_and_tags: CommandCursor[dict[str, Any]] = db.posts.aggregate([
    { "$match" : {"PostTypeId": 1}},
    {"$limit": 1000},

    {'$group':{
            '_id':'$ContentLicense',
            'AllTags': { '$addToSet': "$Tags" } #me añade cada uno de los tags que tiene cada una de las licencias
        }
    }
])
pp(list(license_and_tags))

[{'AllTags': ['<java><android><navigationview>',
              '<licencias><código-abierto>',
              '<android><android-studio>',
              '<php><mysqli>',
              '<python><python-2.7>',
              '<git><git-push>',
              '<python><django>',
              '<php><html><api>',
              '<base-de-datos><cálculo-relacional>',
              '<python><django><django-models><base-de-datos>',
              '<javascript><jquery><asincronismo>',
              '<java><java-swing>',
              '<javascript><hapijs>',
              '<python><django><python-2.7><utf-8><codificación>',
              '<raku>',
              '<c#>',
              '<c#><asp.net><excel>',
              '<python><django><python-3.x>',
              '<java><ordenamiento><lista>',
              '<javascript><google-apps-script><google-hoja-de-cálculo>',
              '<asp.net><asp.net-mvc><librería><bower>',
              '<java><android><android-intent><android-asynctask>',
         

### `$out`

Este comando permite volcar el resultado de un pipeline de agregación en una nueva colección en la base de datos.


Este ejemplo selecciona las publicaciones con `Score` mayor o igual a 40, las asocia con sus usuarios mediante un `$lookup`, extrae el nombre del usuario y almacena el resultado en una nueva colección llamada `stackoverflow_users`.


In [34]:
respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
        { '$match': { 'Score' : {'$gte': 40}}},
        { '$lookup': { #me coje los usuarios
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },
        { '$unwind': '$owner'},
        { '$project' : {
             'username': '$owner.DisplayName'
          }
        },
        {'$out': "stackoverflow_users"}  # me genera una nueva bd que se llama stackoverflow_users
      ])

Un documento de ejemplo de la colección generada `stackoverflow_users`:

In [35]:
pp(db.stackoverflow_users.find_one())

{'_id': ObjectId('69a001966103b90d71f1d877'), 'username': 'Konamiman'}


In [36]:
db.stackoverflow_users.count_documents({})

122

### `$sort`

Este comando permite ordenar todos los documentos en un orden y los devuyelve ordenados. Para ordenar ascendentemente se usa el valor 1, y -1 para ordenar descendentemente.


Este ejemplo selecciona las publicaciones con `Score` mayor o igual a 40 y las ordena en orden ascencente, las asocia con sus usuarios mediante un `$lookup`, extrae el nombre del usuario.

In [37]:
respuestas: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
        { '$match': { 'Score' : {'$gte': 40}}},
         { '$sort': { 'Score' : 1}},
        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },
        { '$unwind': '$owner'},
        { '$project' : {
              'Score': True,
             'username': '$owner.DisplayName'
          }
        }
      ])
respuestas_lst = list(respuestas)
pp(respuestas_lst)

[{'Score': 40,
  '_id': ObjectId('69a001966103b90d71f1d877'),
  'username': 'Konamiman'},
 {'Score': 40,
  '_id': ObjectId('69a001966103b90d71f1ddcf'),
  'username': 'Raider'},
 {'Score': 40,
  '_id': ObjectId('69a001966103b90d71f1de35'),
  'username': 'Gerardo Martínez'},
 {'Score': 40,
  '_id': ObjectId('69a001966103b90d71f1d9c3'),
  'username': 'Paul Vargas'},
 {'Score': 40,
  '_id': ObjectId('69a001976103b90d71f20f10'),
  'username': 'Jordi Castilla'},
 {'Score': 40,
  '_id': ObjectId('69a001976103b90d71f2dec6'),
  'username': 'g3rv4'},
 {'Score': 40,
  '_id': ObjectId('69a001976103b90d71f23673'),
  'username': 'Mariano'},
 {'Score': 40,
  '_id': ObjectId('69a001966103b90d71f1d903'),
  'username': 'fedorqui'},
 {'Score': 40,
  '_id': ObjectId('69a001976103b90d71f23880'),
  'username': 'Mariano'},
 {'Score': 40,
  '_id': ObjectId('69a001986103b90d71f59bec'),
  'username': 'Rutx1'},
 {'Score': 41,
  '_id': ObjectId('69a001976103b90d71f33c50'),
  'username': 'Trauma'},
 {'Score': 41,


## Ejemplos básicos de agregación

### Ejemplo 1: Asociación usuarios con Tags

Con Agregación, vamos a 'construir'(no la construye, solo la muestra) una colección que asocia un usuario con los tags que ha usado en todas sus preguntas.

*Por cuestiones de rendimiento vamos a limitar la agregación a 500 elementos.*

In [38]:
user_tags: CommandCursor[dict[str, Any]] = db.posts.aggregate([
        { "$match" : {"PostTypeId": 1}}, #solo las preguntas

        {"$limit": 500},

        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },

        {'$project':{
            'Tags': True,
            'userid' : {'$arrayElemAt' : ['$owner.Id', 0]},
        }},

        {'$group':{
            '_id':'$userid',
            'AllTags': { '$addToSet': "$Tags" }
        }}
])

user_tags_lst = list(user_tags)
pp(user_tags_lst)

[{'AllTags': ['<python><punteros>', '<python><imagen>', '<bash><verilog>'],
  '_id': 389},
 {'AllTags': ['<c#><asp.net-mvc-5>'], '_id': 2036},
 {'AllTags': ['<javascript><datetime><momentjs><firebase>',
              '<javascript><momentjs><underscorejs>',
              '<javascript><underscorejs>'],
  '_id': 636},
 {'AllTags': ['<c#><.net>', '<c++>', '<c++><c++11>', '<c++><qt>'], '_id': 463},
 {'AllTags': ['<c#><jquery><ajax><asp.net-mvc-5><asp.net-mvc>',
              '<javascript><jquery><typescript>',
              '<typescript><aurelia.io><select2>'],
  '_id': 629},
 {'AllTags': ['<ios><objective-c>'], '_id': 403},
 {'AllTags': ['<java><java-ee><ide>'], '_id': 167},
 {'AllTags': ['<python><pip><pillow>'], '_id': 783},
 {'AllTags': ['<c#><motor-de-juego><unity3d><2d>', '<c#><unity3d>'],
  '_id': 2257},
 {'AllTags': ['<javascript><clausuras>', '<ios><objective-c><uiwebview><ios8>'],
  '_id': 965},
 {'AllTags': ['<git><git-commit>'], '_id': 299},
 {'AllTags': ['<java><android>', '<ph

### Ejemplo 2: Asociación Tags con usuarios

Ahora, dado un ID de tag, qué usuarios han hecho preguntas de ese tag.

*De nuevo limitamos la búsqueda a 500 documentos*.

In [40]:
tags_users: CommandCursor[dict[str, Any]] = db.posts.aggregate( [
        { "$match" : {"PostTypeId": 1}},

        {"$limit": 500},

        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },

        {'$project':{ #proyectamos el Tags y ownerId
            'Tags': True,
            'userid' : {'$arrayElemAt' : ['$owner.Id', 0]},
        }},

        {'$group':{
            '_id':'$Tags',
            'AllUsers': { '$addToSet': "$userid" }
        }}
    ])

tags_users_lst = list(tags_users)
pp(tags_users_lst)

[{'AllUsers': [22], '_id': '<array><perl>'},
 {'AllUsers': [271], '_id': '<latex>'},
 {'AllUsers': [2335, 728, 676, 725], '_id': '<genexus>'},
 {'AllUsers': [608], '_id': '<java><jersey><swagger>'},
 {'AllUsers': [416], '_id': '<matemáticas><coma-flotante><lenguaje-agnóstico>'},
 {'AllUsers': [345], '_id': '<sequelize.js><pruebas-unitarias>'},
 {'AllUsers': [167], '_id': '<java><java-ee><ide>'},
 {'AllUsers': [1827], '_id': '<html5><bootstrap>'},
 {'AllUsers': [1005], '_id': '<django><django-models>'},
 {'AllUsers': [], '_id': '<.net><clickonce><deployment>'},
 {'AllUsers': [23], '_id': '<google-maps><angularjs><jasmine><karma-jasmine>'},
 {'AllUsers': [100], '_id': '<android><cordova><ionic>'},
 {'AllUsers': [305], '_id': '<android><android-layout><unidades-de-medición>'},
 {'AllUsers': [463], '_id': '<c++><qt>'},
 {'AllUsers': [1746], '_id': '<php><mysql><mysqli>'},
 {'AllUsers': [813], '_id': '<vpn><windows>'},
 {'AllUsers': [1788, 729], '_id': '<javascript><html>'},
 {'AllUsers': [

## Ejercicios

### Ejercicio 1: Encontrar las diez primeras preguntas realizadas y mostrar las respuestas aceptadas para las mismas. Mostrar el título de la pregunta y la respuesta dada.

In [41]:
# AcceptedAnswerId: Id de la respuesta aceptada (solo en preguntas). Ej: 4567. Uso: FK a `posts.Id` para marcar aceptadas.

primeras_diez_preguntas :CommandCursor[dict[str,Any]]= db.posts.aggregate([
    {'$match': {'PostTypeId': 1,  #preguntas
              'AcceptedAnswerId': { '$exists': True} #con respuesta aceptada
            }},
    {'$sort': { 'CreationDate': 1 }}, #las ordenamos
    { '$limit': 10 }, #cogemos 10
    {'$lookup': {'from': 'posts',
            'localField': 'AcceptedAnswerId',
            'foreignField': 'Id',
            'as': 'respuesta'}},
    { '$unwind': '$respuesta' },
    {'$project': {'_id': 0,
            'Titulo_Pregunta': '$Title',
            'Cuerpo_Respuesta': '$datos_respuesta.Body'}}
])

pp(list(primeras_diez_preguntas))

[{'Titulo_Pregunta': 'La creación manual de un alias de ruta con un '
                     'DirectRouteProvider personalizado causa un error '
                     '"Multiple actions were found that match the request"'},
 {'Titulo_Pregunta': '¿Es igual utilizar .AsString que .Text para obtener el '
                     'valor de un campo de un DataSet?'},
 {'Titulo_Pregunta': '¿Cómo separar las palabras que contiene un string en '
                     "C++? 'Split '"},
 {'Titulo_Pregunta': '¿Cuándo usar y es seguro emplear const_cast, '
                     'static_cast, dynamic_cast, reinterpret_cast?'},
 {'Titulo_Pregunta': '"Unknown provider: uiGmapGoogleMapApiProvider" al probar '
                     'con Jasmine código que usa el componente de Google Maps '
                     'para AngularJS'},
 {'Titulo_Pregunta': '¿Cómo definir métodos correctamente en TypeScript?'},
 {'Titulo_Pregunta': '"splice() offset past end of array" en diferentes '
                     'versiones de P

In [48]:
# otra forma

primeras_diez_preguntas :CommandCursor[dict[str,Any]]= db.posts.aggregate([
    {'$match': {'PostTypeId': 1}},
    { '$limit': 10 }, #cogemos 10
    {'$lookup': {
            'from': 'posts',
            'localField': 'AcceptedAnswerId',
            'foreignField': 'Id',
            'as': 'respuesta'}},
   { '$project': {
            '_id': 0,
            'Texto_Respuesta': '$Body',
            'Titulo_Pregunta': { '$arrayElemAt': ['$pregunta.Title', 0] }
    }}
])

pp(list(primeras_diez_preguntas))


[{'Texto_Respuesta': '<p>Estoy creando un servicio usando <em>ASP.NET '
                     'WebApi</em>. Quiero añadir soporte para la negociación '
                     'del tipo de contenido basado en extensiones en el '
                     '<em>URI</em>, así que he añadido lo siguiente al código '
                     'de inicialización del '
                     'servicio:</p><br/><br/><pre><code>public static class '
                     'WebApiConfig<br/>{<br/>  public static void '
                     'Register(HttpConfiguration config)<br/>  {<br/>    '
                     'config.Formatters.JsonFormatter.AddUriPathExtensionMapping("json", '
                     '"application/json");<br/>    '
                     'config.Formatters.XmlFormatter.AddUriPathExtensionMapping("xml", '
                     '"application/xml");<br/>  '
                     '}<br/>}<br/></code></pre><br/><br/><p>Para que esto '
                     'funcione necesito crear dos rutas para cada acc

### Ejercicio 2: Encontrar las diez primeras repuestas realizadas y  mostrar la pregunta correspondiente. Mostrar la respuesta dada y el título de la pregunta.



In [49]:
#ParentId FK a posts.Id

primeras_diez_respuestas:CommandCursor[dict[str,Any]]=db.posts.aggregate ([
 {'$match': {'PostTypeId':2}},
 { '$limit': 10 },
 {'$lookup': {
            'from': 'posts',
            'localField': 'ParentId',
            'foreignField': 'Id',
            'as': 'pregunta'
        }},
 { '$project': {
            'Respuesta': '$Body',
            'Titulo_Pregunta': { '$arrayElemAt': ['$pregunta.Title', 0] }
    }}

])
pp(list(primeras_diez_respuestas))

[{'Respuesta': '<p>He encontrado la solución.</p><br/><br/><p>Resulta que las '
               'rutas tienen asignada una <strong>precedencia</strong> '
               'numérica asignada, que el motor de enrutamiento de WebApi usa '
               'para decidir qué ruta usar en caso de conflicto. Las rutas '
               'creadas automáticamente para una misma acción siempre tienen '
               'una precedencia distinta, ¡pero la ruta que yo estaba creando '
               'manualmente tenía la misma precedencia que la ya '
               'existente!</p><br/><br/><p>Así pues la solución es añadir lo '
               'siguiente a <code>GetActionDirectRoutes</code>, inmediatamente '
               'después de <code>new '
               'RouteEntry</code>:</p><br/><br/><pre><code>entry.Route.DataTokens["precedence"] '
               '= <br/>    ((decimal)route.Route.DataTokens["precedence"]) - '
               '0.1M;<br/></code></pre><br/>',
  'Titulo_Pregunta': 'La creación manual 

### Ejercicio 3: Encontrar las 10 preguntas más vistas (`ViewCount`) y muestra el nombre del usuario que las creó.

In [50]:
preguntas_10_masvistas:CommandCursor[dict[str,Any]]=db.posts.aggregate ([
    {'$match': {'PostTypeId':1}},
    {'$sort': { 'ViewCount': -1 }},
    {'$limit' :10},
    {'$lookup': {
        'from': 'users',
        'localField': 'OwnerUserId',
        'foreignField': 'Id',
        'as': 'usuario'}},
    {'$project':{
        'Nombre_usuario': {'$arrayElemAt': ['$usuario.DisplayName', 0]}}}
])

pp(list(preguntas_10_masvistas))

[{'Nombre_usuario': 'Jorgesys', '_id': ObjectId('69a001976103b90d71f262f2')},
 {'Nombre_usuario': 'jasilva', '_id': ObjectId('69a001966103b90d71f1dd9b')},
 {'Nombre_usuario': 'samir Pazo torres',
  '_id': ObjectId('69a001966103b90d71f1eb1e')},
 {'Nombre_usuario': 'Lucas. D', '_id': ObjectId('69a001976103b90d71f1fc4f')},
 {'Nombre_usuario': 'gbianchi', '_id': ObjectId('69a001976103b90d71f26baf')},
 {'Nombre_usuario': 'Roy', '_id': ObjectId('69a001976103b90d71f2316c')},
 {'Nombre_usuario': 'Manuel Rivero Albano',
  '_id': ObjectId('69a001976103b90d71f343cf')},
 {'Nombre_usuario': 'Jordi Castilla',
  '_id': ObjectId('69a001966103b90d71f1d933')},
 {'Nombre_usuario': 'Jordi Castilla',
  '_id': ObjectId('69a001976103b90d71f20f10')},
 {'Nombre_usuario': 'Sergio AG', '_id': ObjectId('69a001976103b90d71f36fc3')}]


### Ejercicio 4: Obtener el resultado de documentos *verbosos* en donde se indique el texto `Body` de una pregunta así como el nombre del usuario que la formuló (`DisplayName`)

In [52]:
# verbosos: textos excesivamente largos
documentos_verbosos : CommandCursor[dict[str,Any]]=db.posts.aggregate ([
{'$sort': { 'Body': -1 }},
{'$lookup': {
        'from': 'users',
        'localField': 'OwnerUserId',
        'foreignField': 'Id',
        'as': 'usuario'}},
{'$project':{
    'Cuerpo' : '$Body',
    'Nombre_usuario' : {'$arrayElemAt':['$usuario.DisplayName',0]}}}

 ])
pp(list(documentos_verbosos))

KeyboardInterrupt: 

### Ejercicio 5: Obtener las respuestas escritas en Enero de 2022 con un `Score` superior a 3.

### Ejercicio 6: Obtener las usuarios que se añadieron al sistema en octubre del año 2017 (`CreationDate`) pero que no hayan realizado después ninguna visita a su perfil (`Views`).

### Ejercicio 7: Obtener un listado, en forma de array, con los id de los usuarios que han contestado cada pregunta. Limita la búsqueda a 2000.

### Ejercicio 8: Repetir el ejercicio anterior, pero obtener el listado en forma de conjunto, con los usuarios que han contestado cada pregunta mostrando para cada usuario su nombre (y no sólo si Id). Limita la búsqueda a 2000.

### Ejercicio 9: Encontrar los 5 usuarios más activos (con más posts) y mostrar su nombre, número de posts y puntuación promedio.

### Ejercicio 10: Analizar los tags más populares: obtener los 10 tags que aparecen en más preguntas, junto con el número de preguntas y la puntuación promedio de las preguntas que los contienen

In [ ]:
print("¡Eso es todo amigos!")